In [1]:
from airsenal.framework.utils import *
from airsenal.framework.FPL_scoring_rules import points_for_yellow_card, points_for_red_card

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
min_minutes = 1
max_minutes = 90
min_matches = 10

query = (
    session.query(PlayerScore)
    .filter(PlayerScore.minutes >= min_minutes)
    .filter(PlayerScore.minutes <= max_minutes)
)
# TODO filter on gw and season
df = pd.read_sql(query.statement, engine)


In [3]:
df["yellow_cards"].value_counts(normalize=True)

0    0.884271
1    0.115729
Name: yellow_cards, dtype: float64

In [4]:
df["red_cards"].value_counts(normalize=True)

0    0.995784
1    0.004216
Name: red_cards, dtype: float64

In [5]:
df["card_pts"] = -df["yellow_cards"] - 3 * df["red_cards"]

In [6]:
card_pts = df.groupby("player_id").card_pts.sum()

match_counts = df.groupby("player_id").card_pts.count()
match_counts[match_counts < min_matches] = min_matches

sum_cards = df.groupby("player_id").card_pts.sum()

avg_cards = sum_cards / match_counts

In [7]:
players = pd.read_sql(session.query(Player).statement, engine)
players.set_index("player_id", inplace=True)

avg_cards = pd.merge(avg_cards, players, how="left", left_index=True, right_index=True)
avg_cards.sort_values(by="card_pts", ascending=True).head(20)

,card_pts,name
player_id,,
956,-0.583333,Marvin Zeegelaar
294,-0.529412,Marcos Rojo
172,-0.466667,Kevin McDonald
1078,-0.454545,Charlie Adam
406,-0.437500,Juan Foyth
588,-0.412698,Jefferson Lerma
2,-0.409091,Sokratis Papastathopoulos
955,-0.400000,Miguel Britos
552,-0.400000,Konstantinos Mavropanos
